### **3rd August** | Scoping out the project

---

#### Objective

Develop a modern data warehouse using SQL Server to consolidate sales data, enabling analytical reporting and informed decision-making.

#### Specifications

- **Data Sources**: Import data from two source systems (ERP and CRM) provided as CSV files.
- **Data Quality**: Cleanse and resolve data quality issues prior to analysis.
- **Integration**: Combine both sources into a single, user-friendly data model designed for analytical queries.
- **Scope**: Focus on the latest dataset only; historization of data is not required.
- **Documentation**: Provide clear documentation of the data model to support both business stakeholders and analytics teams.

#### Medallion Architecture

As per the course’s recommendation, we will be using the Medallion data management paradigm. This layered approach is particularly effective for building a modern data warehouse because it separates raw, cleansed, and curated datasets into clear stages—commonly referred to as Bronze, Silver, and Gold layers. By structuring the pipeline this way, we ensure that each layer has a distinct purpose: Bronze holds unaltered, raw source data; Silver refines and cleanses this data for reliability; and Gold aggregates it into business-ready tables optimised for reporting and analytics.

Opting for Medallion provides several advantages. It improves data quality and trust by ensuring transformations are traceable and reproducible, as all data first lands in a raw state before being standardised and validated. It also enhances maintainability and scalability, allowing us to debug issues at the appropriate layer rather than across the entire warehouse. 

<br>

![data architecture figure 1 image](img/journal_fig1.png)

<br>


| | Bronze Layer | Silver Layer | Gold Layer |
| - | - | - | - |
| **Definition** | Raw, unprocessed data as-is from sources | Clean and standardised data | Business-ready data | 
| **Objective** | Traceability & debugging | (Intermediate layer) Prepare data for analysis | Provide data to be consumed for reporting & analytics |
| **Object Type** | Tables | Tables | Views |
| **Load Method** | Full load (truncate & insert) | Full load (truncate & insert) | None |
| **Data Transformation** | None (as-is) | Data cleaning, standardisation, normalisation, enrichment & derived columns | Data integration, aggregation, business logic & rules |
| **Data Modeling** | None (as-is) | None (as-is) | Start schema, aggregated objects, flat tables |
| **Target Audience** | Data engineers | Data engineers & analysts | Data analysts & business users |  

<br>

#### Layers for Separation of Concerns (SoC)

The above layers mean that we have separation of concerns (SoC) - an important principle where we take a complex system and break it down into independent parts, each focused on a specific responsibility or operation without overlapping with others. So for a data warehouse, SoC means breaking the architecture into independent layers—such as ingestion, transformation, storage, and consumption—so each layer handles its own responsibility without interfering with the others...

<br>

![data architecture figure 2 image](img/journal_fig2.png)

#### Beginnings...

Moving forward, this journal will document each stage of the data warehouse build with clear, structured notes. We will capture:

1. **Data Source Overview** – Key details about each source, including its format, refresh frequency, volume, and method of access.
2. **Data Quality and Validation** – How we identify and resolve issues such as missing data, duplicates, and inconsistencies, as well as checks between Bronze and Silver layers.
3. **Medallion Layer Objectives** – The purpose and responsibilities of the Bronze, Silver, and Gold layers, along with how each supports data trust and reporting readiness.
4. **Target Schema Design** – Sketches and notes on fact and dimension tables, including any decisions around star vs. snowflake schema design.
5. **ETL / ELT Flow** – Steps for moving data between layers, whether through incremental or full loads, and how the process will be orchestrated.
6. **Testing and Monitoring** – Plans for data validation, quality checks, and ongoing monitoring to ensure the reliability of the Gold outputs.
7. **Versioning and Progress Log** – A running journal of decisions, challenges, lessons learned, and key milestones as the project evolves.

#### The high-level goal

![data architecture figure 3 image](img/journal_fig3.png)


### **5th August** | Defining Naming Conventions

---

_(Lifted from the course materials.)_

#### General Principles
- **Naming Conventions**: Use snake_case, with lowercase letters and underscores (_) to separate words.
- **Language**: Use English for all names.
- **Avoid Reserved Words**: Do not use SQL reserved words as object names.

#### Table Naming Conventions

**Bronze Layer Rules**

All names must start with the source system name, and table names must match their original names without renaming.
- `[sourcesystem_entity]`
    - `[sourcesystem]`: Name of the source system (e.g., crm, erp).
    - `[entity]`: Exact table name from the source system.
    - Example: `crm_customer_info` → Customer information from the CRM system.

**Silver Layer Rules**

In this scenario, we are not renaming items between Bronze and Silver. So the rules above will apply in Silver. 

**Gold Layer Rules**

All names must use meaningful, business-aligned names for tables, starting with the category prefix.
- `[category_entity]`
    - `[category]`: Describes the role of the table, such as dim (dimension) or fact (fact table).
    - `[entity]`: Descriptive name of the table, aligned with the business domain (e.g., customers, products, sales).
    - Examples:
        - `dim_customers` → Dimension table for customer data.
        - `fact_sales` → Fact table containing sales transactions.

**Glossary of Category Patterns**
| Pattern | Meaning | Example(s) |
| - | - | - |
| `dim_` | Dimension table | `dim_customer`, `dim_product` | 
| `fact_` | Fact table | `fact_sales` | 
| `agg_` | Aggregated table | `agg_customer`, `agg_sales_monthly` | 


#### Column Naming Conventions

**Surrogate Keys**

- All primary keys in dimension tables must use the suffix `_key`.
- `[table_name]_key`
    - [table_name]: Refers to the name of the table or entity the key belongs to.
    - `_key`: A suffix indicating that this column is a surrogate key.
    - Example: `customer_key` → Surrogate key in the dim_customers table.

**Technical Columns**

- All technical columns must start with the prefix dwh_, followed by a descriptive name indicating the column's purpose.
- `dwh_[column_name]`
    - `dwh`: Prefix exclusively for system-generated metadata.
    - `[column_name]`: Descriptive name indicating the column's purpose.
    - Example: `dwh_load_date` → System-generated column used to store the date when the record was loaded.

**Stored Procedure**

All stored procedures used for loading data must follow the naming pattern: 
- `load_[layer]`.
    - `[layer]`: Represents the layer being loaded, such as bronze, silver, or gold.
    - Example:
        - `load_bronze` → Stored procedure for loading data into the Bronze layer.
        - `load_silver` → Stored procedure for loading data into the Silver layer.
